### Librerías y variables de entorno

In [2]:
import os
import openai
from openai.embeddings_utils import get_embedding, cosine_similarity
import tiktoken

from docx import Document
import pandas as pd

from dotenv import load_dotenv, find_dotenv

In [3]:
# Cargamos las variables de entorno necesarias
load_dotenv()

True

In [4]:
# This example requires environment variables named "OPEN_AI_KEY" and "OPEN_AI_ENDPOINT"
# Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
openai.api_key = os.environ.get('AZURE_OPENAI_KEY')
openai.api_base =  os.environ.get('AZURE_OPENAI_ENDPOINT')
openai.api_type = 'azure'
openai.api_version = '2023-03-15-preview'

# This will correspond to the custom name you chose for your deployment when you deployed a model.
deployment_id='chatv2'
GPT_MODEL= 'gpt-3.5-turbo'
CHAT_GPT_MODEL='chat'


### Definición de funciones

In [5]:
def is_heading(paragraph):
    """Controla el tipo de "heading" del párrafo.

    :param párrafo:
    :return:
    """
    type_heading = ''
    
    if paragraph.style.name.startswith('Heading 1'):
        type_heading = 'heading1'
    elif paragraph.style.name.startswith('Heading 2'):
        type_heading = 'heading2'
    elif paragraph.style.name.startswith('Heading 3'):
        type_heading = 'heading3'
    else:
        type_heading = 'other'
    
    return type_heading

In [1]:
def iterate_document_sections(document):
    """Por cada sección de "headed", genera una secuencia de párrafos.
    Cada secuencia comienza con el párrafo "headed", seguido del texto normal del párrafo.

    :param document: Current Comet Issue
    :return: an article
    """
    paragraphs = [document.paragraphs[0]]
    #paragraphs = []
    for idx, paragraph in enumerate(document.paragraphs[1:]):
        if is_heading(paragraph) == 'heading1':
            yield paragraphs
            paragraphs = [paragraph]
            continue
        elif is_heading(paragraph) == 'heading2':
            yield paragraphs
            paragraphs = [paragraph]
            continue
        elif is_heading(paragraph) == 'heading3':
            yield paragraphs
            paragraphs = [paragraph]
            continue
        else:
            paragraphs.append(paragraph)
    yield paragraphs

In [7]:
def create_document_from_paragraphs(paragraphs):
    """Itera entre los diferentes párrafos, identificando los artículos, y haciendo un split de la documentación.
    El resultado final se almacena en un DF:

    :param paragraphs: Article text
    :return: DF whit single articule for each row.
    """
    new_doc = Document()
    list_contenido = []

    for counter, words in enumerate(paragraphs):
        new_content = words.text
        list_contenido.append(new_content)
        new_doc.add_paragraph(new_content)
    print('----------- FIN PROCESAR NUEVO CONTENIDO-----------')
    text = '\n'.join(list_contenido)
    df_spliteado.loc[len(df_spliteado),'content'] = text

    #new_doc.save(paragraphs[0].text + '.docx')     # esta línea nos permite guardar cada articulo generado en un documento separado

In [8]:
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Devuelve la cantidad de tokens de un string"""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [ ]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    #relatedness_fn=lambda x, y: 1 - openai.spatial.distance.cosine(x, y),
    relatedness_fn=lambda x, y: cosine_similarity(x, y),
    top_n: int = 100
) -> tuple[list[str], list[float]]:
    """Devuelve una lista de artículos relacionados, ordenados del mas relevante al menos relevante."""
    
    query_embedding = get_embedding(query, engine = deployment_id)
    
    strings_and_relatednesses = [
        (row["content"], relatedness_fn(query_embedding, row["vector_embeding"]))
        for i, row in df.iterrows()
    ]
    strings_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    strings, relatednesses = zip(*strings_and_relatednesses)
    return strings[:top_n], relatednesses[:top_n]

In [ ]:
# Ejemplos de prueba
strings, relatednesses = strings_ranked_by_relatedness("tipos de inversiones", df_spliteado, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

In [ ]:
def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Devuelve el msj formateado para GPT, con los artículos relevantes relacionados encontrados."""
    
    strings, relatednesses = strings_ranked_by_relatedness(query, df, top_n=5)
    introduction = 'Utilizar los siguientes artículos sobre los Instructivos y Procesos para seguro de retiro, para contestar las subsecuentes consultas. Si la respuesta no se encuentra entre los artículos, responder "No se encontró ninguna información sobre la consulta."'
    question = f"\n\nConsulta: {query}"
    message = introduction
    for string in strings:
        next_article = f'\n\nArtículos sobre seguros de retiro:\n"""\n{string}\n"""'
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question

In [ ]:
def ask(
    query: str,
    df: pd.DataFrame = df_spliteado,
    model_query: str = GPT_MODEL,
    model: str = CHAT_GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Retorna la respuesta a la consulta realizada, utilizando GPT y los artículos relevantes encontrados."""
    
    message = query_message(query, df, model=model_query, token_budget=token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "Actúa como un asesor que responde consultas sobre Seguros de Retiro. Siempre responder en español."},
        {"role": "user", "content": message},
    ]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    return response_message

# Comenzamos con el "main"

### Procesamiento del Documento a analizar:

- Definimos el documento a procesar.
- Creamos el DataFrame que almacenara los artículos procesados.
- Iteramos todo el documento, completando el DataFrame con los artículos encontrados.

In [ ]:
# Cargamos el Documento a procesar (para este ejemplo, se realiza un analisis a nivel de "headed").
# Definimos el DF que almacenara los datos procesados.

document = Document('./Doc_2.docx')
df_spliteado = pd.DataFrame(columns=['content', 'len_token', 'vector_embeding'])

In [10]:
# iteramos por todos los párrafos del documento, y completamos el DF con la información procesada.

for paragraphs in iterate_document_sections(document):
    create_document_from_paragraphs(paragraphs)

print(df_spliteado.info())
print(df_spliteado.head())


----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- FIN PROCESAR NUEVO CONTENIDO-----------
----------- 

### Análisis de Tokens y Embedding

- Se completa y controla la cantidad de tokens de cada articulo (note: el modelo de Embedding tiene un limite máximo de tokens como input).
- Completamos el DF con los vectores de Embedding correspondiente a cada articulo.

In [11]:
# Calculamos los tokens para cada artículo creado.

df_spliteado['len_token'] = df_spliteado['content'].apply(num_tokens)

In [12]:
df_spliteado.head().sort_values(['len_token'], ascending=False)

,content,len_token,vector_embeding
1,TRATAMIENTO DE LAS COMUNICACIONES\n\nPor solic...,1560,NaN
3,Asesoramiento del Producto\nPrevención Retiro ...,249,NaN
0,Procedimiento - Responsabilidades del puesto\n...,233,NaN
4,\nPodés armarlo a tu medida:\nDecidís en qué m...,115,NaN
2,PREGUNTAS FRECUENTES,8,NaN


In [13]:
# Generamos el vector de Embedding para cada artículo.

df_spliteado['vector_embeding'] = df_spliteado['content'].apply(lambda x: get_embedding(x, engine = deployment_id))

In [14]:
df_spliteado.head()

,content,len_token,vector_embeding
0,Procedimiento - Responsabilidades del puesto\n...,233,"[-0.017123669385910034, 0.0010063608642667532,..."
1,TRATAMIENTO DE LAS COMUNICACIONES\n\nPor solic...,1560,"[-0.012864670716226101, 0.0014471879694610834,..."
2,PREGUNTAS FRECUENTES,8,"[0.005458911415189505, -0.017592333257198334, ..."
3,Asesoramiento del Producto\nPrevención Retiro ...,249,"[-0.011354345828294754, -0.024713190272450447,..."
4,\nPodés armarlo a tu medida:\nDecidís en qué m...,115,"[-0.006575466133654118, -0.024058355018496513,..."


### Ejemplos de consultas realizadas:

In [61]:
ask('¿En que se invierte mi dinero?')

'La Compañía invierte el dinero en Obligaciones Negociables, Fondos comunes de inversión, títulos públicos, plazos fijos y otros, según lo determinado por la SSN y dentro del porcentaje máximo permitido.'